### Парсинг HTML. XPath

1)Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news
Для парсинга использовать xpath. Структура данных должна содержать:
название источника,
наименование новости,
ссылку на новость,
дата публикации

2)Сложить все новости в БД

In [135]:
from pprint import pprint
from lxml import html
import requests
import pandas as pd
from datetime import datetime
import re

header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'
    }

In [233]:
def get_news_lenta_ru():
    
    news = []
    
    main_link = 'https://lenta.ru'
    
    response = requests.get(main_link, headers = header)
    dom = html.fromstring(response.text)
    blocks = dom.xpath("//section[@class='row b-top7-for-main js-top-seven'] // div[@class='first-item'] | //section[@class='row b-top7-for-main js-top-seven'] // div[@class='item']")
    
    for block in blocks:
        item = {}
        item['news_resource'] = 'LENTA.RU'
        news_name = block.xpath('.//h2//a/text() | .//a/text()')
        for i in range(len(news_name)):
            news_name[i] = news_name[i].replace(u'\xa0', u' ')        
        item['news_name'] = news_name
        news_link = block.xpath('.//h2//a/@href | .//a/@href')
        for i in range(len(news_link)):
            news_link[i] = main_link + news_link[i]
        item['news_link'] = news_link      
        news_date = block.xpath('.//h2//a/time[@class="g-time"]/@datetime | .//a/time[@class="g-time"]/@datetime')
        item['news_data'] = news_date
        
        news.append(item)
    
    return news

In [234]:
get_news_lenta_ru()

[{'news_resource': 'LENTA.RU',
  'news_name': ['Возле границы Северной и Южной Кореи раздался взрыв',
   'Перейти в «Мою Ленту»'],
  'news_link': ['https://lenta.ru/news/2020/06/16/hlo_pok/',
   'https://lenta.ru/news/2020/06/16/hlo_pok/',
   'https://lenta.ru/auth/',
   'https://lenta.ru/my/dashboard'],
  'news_data': [' 09:57, 16 июня 2020']},
 {'news_resource': 'LENTA.RU',
  'news_name': ['В ДНР и ЛНР проголосуют по поправкам к Конституции России'],
  'news_link': ['https://lenta.ru/news/2020/06/16/vsenarodnoye/'],
  'news_data': [' 10:25, 16 июня 2020']},
 {'news_resource': 'LENTA.RU',
  'news_name': ['Напавший на российских полицейских стрелок вел дневник и был замкнут'],
  'news_link': ['https://lenta.ru/news/2020/06/16/dnevnik/'],
  'news_data': [' 10:25, 16 июня 2020']},
 {'news_resource': 'LENTA.RU',
  'news_name': ['Лидера воровского мира Абхазии обвинили в мелком хулиганстве'],
  'news_link': ['https://lenta.ru/news/2020/06/16/vorvzakone/'],
  'news_data': [' 10:15, 16 июня 

In [396]:
def get_news_yandex_ru():
    
    news = []
    
    main_link = 'https://yandex.ru'
    
    response = requests.get(main_link + '/news/', headers = header)
    dom = html.fromstring(response.text)
    blocks = dom.xpath("//div[@class='story__content'] | //td[@class='stories-set__item']")
    
    for block in blocks:
        item = {}
        news_info = block.xpath(".//div[@class='story__date']/text()")
        for i in range(len(news_info)):
            news_date = re.findall('\d{2}:\d{2}', news_info[i])
            news_resource = re.sub('\d{2}:\d{2}', '', news_info[i])
        item['news_resource'] = news_resource
        news_name = block.xpath(".//h2[@class='story__title']//a/text()")
        item['news_name'] = news_name
        news_link = block.xpath(".//h2[@class='story__title']//a/@href")
        for i in range(len(news_link)):
            news_link[i] = main_link + news_link[i]
        item['news_link'] = news_link
        item['news_data'] = news_date
        
        news.append(item)
    
    return news

In [397]:
get_news_yandex_ru()

[{'news_resource': 'Известия ',
  'news_name': ['Сенатор предложил в 2021 году сократить новогодние каникулы'],
  'news_link': ['https://yandex.ru/news/story/Senator_predlozhil_v_2021_godu_sokratit_novogodnie_kanikuly--6b785b775526364bb889e0e0fe88d838?lr=2&lang=ru&stid=lXHOO-j_WOe9YANCwARp&persistent_id=102685321&rubric=index&from=index'],
  'news_data': ['14:55']},
 {'news_resource': 'Российская газета ',
  'news_name': ['В центре Назрани произошла перестрелка'],
  'news_link': ['https://yandex.ru/news/story/V_centre_Nazrani_proizoshla_perestrelka--dd53f570a3455cf4f0ee90853b72f331?lr=2&lang=ru&stid=kKkzsbjYg5kXZPHIHNcJ&persistent_id=102706694&rubric=index&from=index'],
  'news_data': ['14:54']},
 {'news_resource': 'Газета.Ru ',
  'news_name': ['В Госдуму внесли законопроект об удаленной работе'],
  'news_link': ['https://yandex.ru/news/story/V_Gosdumu_vnesli_zakonoproekt_ob_udalennoj_rabote--42e0ff8edd3d1b25961e2435f31f915b?lr=2&lang=ru&stid=SBSvKMfJ9RErrC7voHoc&persistent_id=10265904

In [398]:
def get_news_mail_ru():
    
    news = []
    
    main_link = 'https://news.mail.ru/'
    response = requests.get(main_link, headers = header)
    dom = html.fromstring(response.text)
    blocks = dom.xpath('//div[@class="block"]//div[@class="daynews__item daynews__item_big"] | //div[@class="block"]//div[@class="daynews__item"]')

    for block in blocks:
        item = {}
        news_link = block.xpath('./a/@href')
                
         
        
        for i in range(len(news_link)):
            news_link[i] = main_link + news_link[i]
            
            temp_response = requests.get(news_link[i], headers = header)
            temp_dom = html.fromstring(temp_response.text)
            temp_blocks = temp_dom.xpath('//div[@class="cols__inner"]')
            for temp_block in temp_blocks:
                news_name = temp_block.xpath('//h1/text()')
                news_date = temp_block.xpath('//span[@class="note__text breadcrumbs__text js-ago"]/@datetime')
                news_resource = temp_block.xpath('//span[@class="breadcrumbs__item"]//a//span[@class="link__text"]/text()')
    
        item['news_resource'] = news_resource
        item['news_name'] = news_name
        item['news_link'] = news_link
        item['news_data'] = news_date
            
        news.append(item)
    
    return news

In [399]:
get_news_mail_ru()

[{'news_resource': ['Ведомости'],
  'news_name': ['В России выявили 8248 новых случаев заражения коронавирусом'],
  'news_link': ['https://news.mail.ru//incident/42203003/'],
  'news_data': ['2020-06-16T10:39:26+03:00']},
 {'news_resource': ['BBC News Русская служба'],
  'news_name': ['Сеул: власти КНДР взорвали штаб по взаимодействию с Южной Кореей'],
  'news_link': ['https://news.mail.ru//incident/42204424/'],
  'news_data': ['2020-06-16T13:58:45+03:00']},
 {'news_resource': ['Секрет Фирмы'],
  'news_name': ['Россиян оштрафовали на 400 млн рублей за нарушения режима самоизоляции'],
  'news_link': ['https://news.mail.ru//incident/42203281/'],
  'news_data': ['2020-06-16T14:32:26+03:00']},
 {'news_resource': ['Газета Коммерсантъ'],
  'news_name': ['Не признан — не газ'],
  'news_link': ['https://news.mail.ru//economics/42198167/'],
  'news_data': ['2020-06-16T09:52:55+03:00']},
 {'news_resource': ['Новости Mail.ru'],
  'news_name': ['Показаны лучшие фото с высоты птичьего полета'],
  '

In [400]:
def parser_news():
        
    news_date = []
    news_date.extend(get_news_lenta_ru())
    news_date.extend(get_news_yandex_ru())
    news_date.extend(get_news_mail_ru())
    
    df = pd.DataFrame(news_date)

    return df

In [401]:
df = parser_news()
df

,news_resource,news_name,news_link,news_data
0,LENTA.RU,[Раскрыта продолжительность действия вакцины о...,"[https://lenta.ru/news/2020/06/16/vaktsina/, h...","[ 15:05, 16 июня 2020]"
1,LENTA.RU,[Александра Емельяненко раскритиковали за опас...,[https://lenta.ru/news/2020/06/16/aleksemelyac...,"[ 15:07, 16 июня 2020]"
2,LENTA.RU,[«Газпром» начал демонтаж труб для транзита че...,[https://lenta.ru/news/2020/06/16/ne_nuzhen/],"[ 15:06, 16 июня 2020]"
3,LENTA.RU,[Москвичам предложили работать с 6 утра ],[https://lenta.ruhttps://moslenta.ru/news/mosk...,"[ 15:00, 16 июня 2020]"
4,LENTA.RU,[Лукашенко призвал брать пример с советских вр...,[https://lenta.ru/news/2020/06/16/o_tempora_o_...,"[ 14:58, 16 июня 2020]"
...,...,...,...,...
75,[Ведомости],[В России выявили 8248 новых случаев заражения...,[https://news.mail.ru//incident/42203003/],[2020-06-16T10:39:26+03:00]
76,[BBC News Русская служба],[Сеул: власти КНДР взорвали штаб по взаимодейс...,[https://news.mail.ru//incident/42204424/],[2020-06-16T13:58:45+03:00]
77,[Секрет Фирмы],[Россиян оштрафовали на 400 млн рублей за нару...,[https://news.mail.ru//incident/42203281/],[2020-06-16T14:32:26+03:00]
78,[Газета Коммерсантъ],[Не признан — не газ],[https://news.mail.ru//economics/42198167/],[2020-06-16T09:52:55+03:00]


In [405]:
from pymongo import MongoClient

In [408]:
def save_to_db(data_frame, db_name):
    
    try:
        client = MongoClient('mongodb://127.0.0.1:27017')
        db = client[db_name]
        
        data = data_frame.to_dict('records')
        
        for item in data:
            db.newsdb.insert_one(item)
        
        return f'Data saved to: {db_name}'
        
    except:
    
        return 'Something go wrong.'

In [411]:
news = 'News'
save_to_db(df, news)

'Data saved to: News'